# Neural Network Prediction of Rössler System Dynamics

**Team 19**: Vlad-Flavius Misăilă, Robert-Daniel Man, Sebastian-Adrian Mărginean

## Overview

The Rössler system is a continuous-time dynamical system that exhibits chaotic behavior with a simpler structure than the Lorenz system:

$$
\begin{align}
\frac{dx}{dt} &= -y - z \\
\frac{dy}{dt} &= x + ay \\
\frac{dz}{dt} &= b + z(x - c)
\end{align}
$$

**Standard parameters**: $a = 0.2$, $b = 0.2$, $c = 5.7$

**Objective**: Predict Rössler system trajectories using neural networks and compare with Lorenz system.

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import torch

from src.dynamical_systems import RosslerSystem
from src.data_preparation import generate_trajectory, create_sequences
from src.neural_models import FeedForwardPredictor, LSTMPredictor, NeuralPredictor
from src.evaluation import (
    evaluate_prediction, plot_trajectory_3d, plot_time_series,
    plot_training_history, prediction_horizon_analysis
)

np.random.seed(42)
torch.manual_seed(42)
plt.rcParams['figure.dpi'] = 100
%matplotlib inline

print("✓ Setup complete!")

## 1. Generate Rössler System Data

In [ ]:
# Initialize Rössler system
rossler = RosslerSystem(a=0.2, b=0.2, c=5.7)

# Generate trajectory
initial_state = np.array([1.0, 1.0, 1.0])
t, trajectory = generate_trajectory(
    rossler,
    initial_state=initial_state,
    t_span=(0, 200),  # Longer time for Rössler
    dt=0.01,
    noise_std=0.0
)

print(f"✓ Generated {len(t)} time points")
print(f"✓ Trajectory shape: {trajectory.shape}")

## 2. Visualize the Rössler Attractor

In [ ]:
# 3D visualization
fig = plt.figure(figsize=(15, 5))

# 3D plot
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], 
         'b-', alpha=0.6, linewidth=0.5)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('Z')
ax1.set_title('Rössler Attractor (3D)')

# X-Y projection
ax2 = fig.add_subplot(132)
ax2.plot(trajectory[:, 0], trajectory[:, 1], 'b-', alpha=0.6, linewidth=0.5)
ax2.set_xlabel('X')
ax2.set_ylabel('Y')
ax2.set_title('X-Y Projection')
ax2.grid(True, alpha=0.3)

# X-Z projection (shows characteristic band structure)
ax3 = fig.add_subplot(133)
ax3.plot(trajectory[:, 0], trajectory[:, 2], 'b-', alpha=0.6, linewidth=0.5)
ax3.set_xlabel('X')
ax3.set_ylabel('Z')
ax3.set_title('X-Z Projection')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Time series
plot_time_series(
    t[:2000],
    trajectory[:2000],
    feature_names=['X', 'Y', 'Z'],
    title='Rössler System Time Series'
)

## 3. Prepare Data and Train Models

In [ ]:
# Prepare sequences
WINDOW_SIZE = 50
X_train, y_train, X_test, y_test, scaler = create_sequences(
    trajectory, window_size=WINDOW_SIZE, train_ratio=0.8, normalize=True
)

print(f"Training: X={X_train.shape}, y={y_train.shape}")
print(f"Testing: X={X_test.shape}, y={y_test.shape}")

In [ ]:
# Train Feed-Forward Network
print("Training Feed-Forward Network...")
fnn = FeedForwardPredictor(WINDOW_SIZE * 3, [128, 64, 32], 3, dropout=0.1)
fnn_predictor = NeuralPredictor(fnn, learning_rate=0.001)
fnn_history = fnn_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=64)

plot_training_history(fnn_history, 'FNN Training (Rössler)')

In [ ]:
# Train LSTM Network
print("Training LSTM Network...")
lstm = LSTMPredictor(3, 64, 2, 3, dropout=0.1)
lstm_predictor = NeuralPredictor(lstm, learning_rate=0.001)
lstm_history = lstm_predictor.train(X_train, y_train, X_test, y_test, epochs=100, batch_size=64)

plot_training_history(lstm_history, 'LSTM Training (Rössler)')

## 4. Evaluate Models

In [ ]:
# One-step predictions
fnn_pred = fnn_predictor.predict(X_test)
lstm_pred = lstm_predictor.predict(X_test)

fnn_metrics = evaluate_prediction(y_test, fnn_pred)
lstm_metrics = evaluate_prediction(y_test, lstm_pred)

print("="*60)
print("ONE-STEP PREDICTION RESULTS (Rössler System)")
print("="*60)
print(f"\nFeed-Forward: RMSE={fnn_metrics['rmse']:.6f}, MAE={fnn_metrics['mae']:.6f}")
print(f"LSTM:         RMSE={lstm_metrics['rmse']:.6f}, MAE={lstm_metrics['mae']:.6f}")
print("="*60)

In [ ]:
# Visualize predictions
y_test_orig = scaler.inverse_transform(y_test)
fnn_pred_orig = scaler.inverse_transform(fnn_pred)
lstm_pred_orig = scaler.inverse_transform(lstm_pred)

plot_trajectory_3d(y_test_orig[:500], fnn_pred_orig[:500],
                   title='FNN: Rössler One-Step Prediction')

plot_trajectory_3d(y_test_orig[:500], lstm_pred_orig[:500],
                   title='LSTM: Rössler One-Step Prediction')

## 5. Multi-Step Prediction

In [ ]:
# Iterative prediction
initial_window = X_test[0]
n_steps = 300
true_future = y_test[:n_steps]

fnn_future = fnn_predictor.iterative_predict(initial_window, n_steps)
lstm_future = lstm_predictor.iterative_predict(initial_window, n_steps)

# Visualize
plot_trajectory_3d(
    scaler.inverse_transform(true_future),
    scaler.inverse_transform(fnn_future),
    title=f'FNN: {n_steps}-Step Prediction (Rössler)'
)

plot_trajectory_3d(
    scaler.inverse_transform(true_future),
    scaler.inverse_transform(lstm_future),
    title=f'LSTM: {n_steps}-Step Prediction (Rössler)'
)

## 6. Prediction Horizon Analysis

In [ ]:
# Error growth analysis
fnn_errors, steps = prediction_horizon_analysis(
    fnn_predictor, initial_window, true_future, max_steps=n_steps
)
lstm_errors, _ = prediction_horizon_analysis(
    lstm_predictor, initial_window, true_future, max_steps=n_steps
)

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(steps, fnn_errors, 'b-', linewidth=2, label='FNN', alpha=0.7)
ax.plot(steps, lstm_errors, 'r-', linewidth=2, label='LSTM', alpha=0.7)
ax.set_xlabel('Prediction Horizon (steps)', fontsize=12)
ax.set_ylabel('RMSE', fontsize=12)
ax.set_title('Prediction Error Growth (Rössler System)', fontsize=14)
ax.legend(fontsize=12)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')
plt.show()

## 7. Summary

### Observations:

1. **Rössler vs Lorenz**: The Rössler system has a simpler structure (single band) compared to Lorenz (double-lobed)
2. **Predictability**: Both systems show exponential error growth, but characteristics differ
3. **Model Performance**: LSTM typically outperforms FNN for longer prediction horizons
4. **Chaotic Nature**: Limited prediction horizon is a fundamental property, not a model limitation

### Key Insights:

- The Rössler attractor's continuous band structure makes it slightly easier to predict than Lorenz
- Neural networks successfully learn the underlying dynamics without explicit equations
- Recurrent architectures (LSTM) better capture temporal dependencies

In [ ]:
print("\n✓ Rössler system analysis complete!")